In [8]:
import re
from collections import Counter

fp = open("quotes.txt", "r")
file = fp.readlines()

In [2]:
#strip line breaks
removeBreak = [line[:-1] for line in file]
#combine quote and name (answer to part a)
quotes = [removeBreak[2*i] + " - " + removeBreak[2*i+1] for i in range(len(removeBreak)/2)]
print quotes[0]

How we spend our days is, of course, how we spend our lives. - Annie Dillard


In [5]:
#answer to part b
#split_words contains the split quotes provided, below is a function to take a general quote
def splitQuote(quote):
    quote = quote.lower()
    quote = re.sub("[`]", '', quote)
    quote = re.split('\s+', re.sub('[^\w]', ' ', quote))
    return quote

split_words = map(splitQuote, quotes)
print split_words[6]

['the', 'heart', 'has', 'its', 'reasons', 'of', 'which', 'the', 'mind', 'knows', 'nothing', 'blaise', 'pascal']


In [15]:
#part c, creat postings-list dictionary
pld = {}
for i in range(len(quotes)):
    pld[quotes[i]] = dict(Counter(split_words[i]))

In [16]:
pld["The heart has its reasons, of which the mind knows nothing. - Blaise Pascal"]

{'blaise': 1,
 'has': 1,
 'heart': 1,
 'its': 1,
 'knows': 1,
 'mind': 1,
 'nothing': 1,
 'of': 1,
 'pascal': 1,
 'reasons': 1,
 'the': 2,
 'which': 1}

In [17]:
#part d

#create a list of all words that appear in quotes.txt
allWords = [dicts.keys() for dicts in pld.values()]
allWords = list(set().union(*allWords))
dictionary = sorted(allWords)

In [27]:
#populate the keys of reverse postings-list dictionary with all possible words
rpld = {}
for entry in dictionary: rpld[entry] = {}

In [34]:
#populate values of rpld with a dictionary of quotes in which the keyword appears
for words in dictionary:
    for key, value in pld.iteritems():
        if words in value:
            tempdict = {key : value[words]}
            rpld[words].update( {key:value[words]})

#example
print rpld['entertainer']

{'An actor is at most a poet and at least an entertainer. - Marlon Brando': 1}


In [55]:
#part e, create TF-IDF function
import math
def TFIDF(fullQuote, word):
    word = word.lower() 
    try:
        tf = float(pld[fullQuote][word]) / max(pld[fullQuote].values())
        idf = math.log10(len(pld)/float(len(rpld[word].keys())))
        tfidf = tf * idf
        return tfidf
    except KeyError:
        return 0

In [56]:
TFIDF('An actor is at most a poet and at least an entertainer. - Marlon Brando', 'entertainer')

1.4737168609435254

In [57]:
#part f, function takes a word and returns TF-IDF score of each quote containing the word
def singleWord(word):
    returnDict = {}
    matches = list(rpld[word])
    for quote in matches:
        returnDict[quote] = TFIDF(quote, word)
    return returnDict

In [58]:
singleWord('marlon')

{'An actor is at most a poet and at least an entertainer. - Marlon Brando': 1.235156233583694,
 'Never confuse the size of your paycheck with the size of your talent. - Marlon Brando': 1.235156233583694,
 'To grasp the full significance of life is the actor`s duty, to interpret it is his problem and to express it his dedication. - Marlon Brando': 0.8234374890557961}

In [44]:
#part g, function takes a list of words and returns the TF-IDF score for every quote with at least one match
def multWord(wordList):
    matches={}
    for words in wordList:
        matches.update(rpld[words])
    matches = list(matches) 
    
    returnDict = {}
    for quote in matches:
        value = 0
        for words in wordList:
            value += TFIDF(quote, words)
        returnDict[quote] = value
    print returnDict    

In [62]:
multWord(['entertainer', 'actor', 'film'])

{'The movie`s director is the pilot. It`s his vision. For an actor, the time to worry about flying is when you`re on the ground. If you don`t want to fly with the director, don`t get on the plane. - Denzel Washington': 0.4410672877038449, 'An actor is at most a poet and at least an entertainer. - Marlon Brando': 2.79691872405506, 'Film lovers are sick people. - Francois Truffaut': 2.9474337218870508}
